# What is Docker?

Lets download and find out ;)

## 1
[Download Docker](https://download.docker.com/win/stable/InstallDocker.msi)
and install then skip to step 2.


In [10]:
$$async$$ = true;
var path = require('path');
var http = require('https');
var fs = require('fs');
var exec = require('child_process').exec;
var elevateLoc = path.join(process.cwd(), 'selenium-act', 'elevate.zip');
var expandedLoc = path.join(process.cwd(), 'selenium-act', 'elevate');
var elevateExecLoc = path.join(process.cwd(), 'selenium-act', 'elevate', 'bin', 'x86', 'Release', 'elevate.exe');
var dockerLoc = path.join(process.cwd(), 'selenium-act', 'InstallDocker.msi');
var logLoc = path.join(process.cwd(), 'selenium-act', 'log.txt');
console.log('checking if Docker is installed');
var docker = exec('docker ps', function (err, stdout, stderr) {
    //if (stdout.indexOf('not found') > -1) {
        console.log('Docker not found, installing');
        if (process.platform === 'win32') {
            console.log('downloading Docker for Windows');
            var downloads = http.get('https://download.docker.com/win/stable/InstallDocker.msi', (r) => {
                r.pipe(fs.createWriteStream(dockerLoc)).on('finish', () => {
                    console.log('downloading jpassing/elevate');
                    http.get('https://github.com/jpassing/elevate/releases/download/1.0/elevate.zip', (r) => {
                        http.get(r.headers['location'], (r) => {
                            r.pipe(fs.createWriteStream(elevateLoc)).on('finish', () => {
                                console.log('downloaded docker.msi, elevate.zip');
                                console.log('extracting elevate.zip');
                                var expand = exec('powershell -c "Expand-Archive -Force ' + elevateLoc + ' ' + expandedLoc + '"', () => {
                                    console.log('running Docker install')
                                    var install = exec(elevateExecLoc + ' powershell -c "new-netfirewallrule -displayname "docker engine" -direction inbound -action allow -protocol tcp -LocalPort 2375" msiexec /i ' + dockerLoc + ' /qn /L*V! ' + logLoc, () => {
                                        $$done$$('installed Docker');
                                    });
                                    install.stdout.on('data', (d) => console.log(d));
                                    install.stderr.on('data', (d) => console.log(d));
                                });
                                expand.stdout.on('data', (d) => console.log(d));
                                expand.stderr.on('data', (d) => console.log(d));
                            });
                        });
                    });
                });
            });
        }
    //} else {
    //    $$done$$('Docker is already installed');
    //}
});
docker.stdout.on('data', (d) => console.log(d));
docker.stderr.on('data', (d) => console.log(d));


checking if Docker is installed
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                                                                                                                                            NAMES
199004cbb085        act-selenium
        "/opt/bin/entry_po..."   About an hour ago   Up About an hour    0.0.0.0:3000->3000/tcp, 0.0.0.0:4200->4200/tcp, 0.0.0.0:4444->4444/tcp, 0.0.0.0:5900->5900/tcp, 0.0.0.0:6080->6080/tcp, 0.0.0.0:8888->8888/tcp   act-selenium

Docker not found, installing
downloading Docker for Windows
downloading jpassing/elevate
downloaded docker.msi, elevate.zip
extracting elevate.zip
running Docker install
T h e   s y s t e m   a d m i n i s t r a t o r   h a s   s e t   p o l i c i e s   t o   p r e v e n t   t h i s   i n s t a l l a t i o n . 
 


'installed Docker'

Create a Docker container with some dev tools:
* selenium (xvfb, x11vnc, novnc, mocha)
* source code (git, act.ecommerce, angular-cli)

Start from the selenium-chrome-debug image

In [2]:
$$async$$ = true;
var path = require('path');
var dockerfilePath = path.join(process.cwd(), 'selenium-act', 'Dockerfile');

var aptPart = `
FROM selenium/standalone-chrome-debug

RUN apt-get -qq update
RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make 
RUN wget -O - https://deb.nodesource.com/setup_7.x | bash
RUN apt-get install -y nodejs
RUN nodejs -v

`;

var nodePart = `

ENV NODE_TLS_REJECT_UNAUTHORIZED 0
RUN npm install -g live-server babel-cli concurrently node-gyp nativescript@latest

COPY act.ecommerce /home/seluser/act.ecommerce
COPY mobile /home/seluser/mobile
ADD novnc /home/seluser/novnc

WORKDIR /home/seluser/act.ecommerce
RUN npm install
RUN npm run build

WORKDIR /home/seluser/mobile
RUN npm install

WORKDIR /home/seluser/
# modify the entry_point from selenium-docker to run our extra services
RUN dos2unix /home/seluser/novnc/utils/launch.sh
RUN sed -i '/wait \$NODE_PID/i/home/seluser/novnc/utils/launch.sh --vnc localhost:5900 &' /opt/bin/entry_point.sh
RUN sed -i '/wait \$NODE_PID/inpm --prefix /home/seluser/mobile run static &' /opt/bin/entry_point.sh

EXPOSE 4200
EXPOSE 4444
EXPOSE 3000

`;
var output = aptPart + nodePart;
var http = require('http');
// check if there is a local-npm server running on Brian's machine
http.get('http://SCO-100001269.ad.swiftpage.com:5080', (r) => {
    if (r.statusCode === 200) {
        console.log('local-npm found!');
        output = aptPart + `
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set registry http://sco-100001269.ad.swiftpage.com:5080
RUN npm config set strict-ssl false
` + nodePart;
    }
    var fs = require('fs');
    fs.writeFile(dockerfilePath, output, function () {
        $$done$$('saved Dockerfile')
    });
});



local-npm found!


'saved Dockerfile'

## 2

Create a Docker file and project directory.

In [3]:
/* Your directory structure should look like this
file -What is Docker.ipynb
dir  -selenium-act
file |--Dockerfile
dir  |--act.ecommerce (from git)
dir  |--mobile       (git repo in progress)
dir  |--novnc        (https://github.com/novnc/noVNC/archive/master.zip)
*/
$$async$$ = true;
var path = require('path');
var seleniumPath = path.join(process.cwd(), 'selenium-act');
var novncDir = path.join(seleniumPath, 'novnc');
var actDir = path.join(seleniumPath, 'act.ecommerce');
var seleniumTestDir = path.join(seleniumPath, 'selenium-tests');
var githubActions = function () {
    try {
        fs.mkdirSync(seleniumPath);
    } catch (err) {
        if (err.code !== 'EEXIST') throw err;
    }
    var rimraf = require('rimraf');
    rimraf(novncDir, function () {
        var cloneCmd = exec('git clone https://github.com/novnc/noVNC.git ' + novncDir, () => {
            rimraf(actDir, function () {
                var cloneActCmd = exec('git clone -b Buy_funnel https://github.com/Swiftpage/act.ecommerce.git ' + actDir, () => {
                    $$done$$('cloned act.ecommerce, selenium-tests, and novnc');
                });
                cloneActCmd.stdout.on('data', (d) => console.log(d));
                cloneActCmd.stderr.on('data', (d) => console.log(d));
            });
        });
        cloneCmd.stdout.on('data', (d) => console.log(d));
        cloneCmd.stderr.on('data', (d) => console.log(d));
    });
};
var exec = require('child_process').exec;
try {
    require.resolve("rimraf");
    githubActions();
} catch(e) {
    require.resolve("npm");
    var installCmd = exec('npm install -g rimraf', () => {
        githubActions();
    });
    installCmd.stdout.on('data', (d) => console.log(d));
    installCmd.stderr.on('data', (d) => console.log(d));
}



Cloning into 'C:\Users\brian.cullinan\Documents\jupytangular2\notebooks\selenium-act\novnc'...

Cloning into 'C:\Users\brian.cullinan\Documents\jupytangular2\notebooks\selenium-act\act.ecommerce'...



'cloned act.ecommerce, selenium-tests, and novnc'

## 3
Build the Docker image

In [4]:
$$async$$ = true;
var exec = require('child_process').exec;
var path = require('path');
var seleniumPath = path.join(process.cwd(), 'selenium-act');
var buildCmd = exec('docker build -t act-selenium ' + seleniumPath, () => {
    var imagesCmd = exec('docker images', () => {
        $$done$$('built Docker container')
    });
    imagesCmd.stdout.on('data', (d) => console.log(d));
    imagesCmd.stderr.on('data', (d) => console.log(d));
});
buildCmd.stdout.on('data', (d) => console.log(d));
buildCmd.stderr.on('data', (d) => console.log(d));





Step 1/27 : FROM selenium/standalone-chrome-debug

 ---> 870685c556e8
Step 2/27 : RUN apt-get -qq update

 ---> Using cache
 ---> aef2262dcd46
Step 3/27 : RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make

 ---> Using cache
 ---> 8fff18fb3618

Step 4/27 : RUN wget -O - https://deb.nodesource.com/setup_7.x | bash

 ---> Using cache

 ---> 30be9f744772
Step 5/27 : RUN apt-get install -y nodejs

 ---> Using cache

 ---> b60411495423
Step 6/27 : RUN nodejs -v

 ---> Using cache
 ---> 9cb2b0e27b27
Step 7/27 : RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080

 ---> Using cache

 ---> b759ef2daf12
Step 8/27 : RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080

 ---> Using cache

 ---> 372dda5d235a
Step 9/27 : RUN npm config set registry http://sco-100001269.ad.swiftpage.com:5080

 ---> Using cache

 ---> 41cc0747e104
Step 10/27 : RUN npm config set strict-ssl false

 ---> Using cache
 ---> 00b3a3ecdfb1
Ste

'built Docker container'

## 4

Run your Docker container

In [5]:
$$async$$ = true;
var exec = require('child_process').exec;
console.log('stopping current Docker');
var stopCmd = exec('docker stop act-selenium', () => {
    console.log('remove current Docker');
    var rmCmd = exec('docker rm act-selenium', () => {
        console.log('running new Docker');
        var runCmd = exec('docker run --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 act-selenium', () => {
            console.log('checking new Docker is running');
            var psCmd = exec('docker ps', () => {
                $$done$$('launched Docker container');
            });
            psCmd.stdout.on('data', (d) => console.log(d));
            psCmd.stderr.on('data', (d) => console.log(d));
        });
        runCmd.stdout.on('data', (d) => console.log(d));
        runCmd.stderr.on('data', (d) => console.log(d));
    });
    rmCmd.stdout.on('data', (d) => console.log(d));
    rmCmd.stderr.on('data', (d) => console.log(d));
});
stopCmd.stdout.on('data', (d) => console.log(d));
stopCmd.stderr.on('data', (d) => console.log(d));



stopping current Docker
act-selenium

remove current Docker
act-selenium

running new Docker
199004cbb085dd7477c37ed6f31ea93cf6457df9cb90353d976e7fca13d4e718

checking new Docker is running
CONTAINER ID        IMAGE               COMMAND                  CREATED                  
STATUS                  PORTS                                                                                                                                            NAMES
199004cbb085        act-selenium        "/opt/bin/entry_po..."   Less than a second ago   Up Less than a second   0.0.0.0:3000->3000/tcp, 0.0.0.0:4200->4200/tcp, 0.0.0.0:4444->4444/tcp, 0.0.0.0:5900->5900/tcp, 0.0.0.0:6080->6080/tcp, 0.0.0.0:8888->8888/tcp   act-selenium



'launched Docker container'

## 5

Connect to VNC so we can see the test play out

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true) in a separate browser

In [8]:
// wait a few seconds for the container services to start
$$async$$ = true;
setTimeout(() => {
$$.html(
    '<div style="display:block; width:100%; padding-bottom:75%;position:relative;">' +
    '<iframe id="vnc"' +
        'style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;"' +
        'src="http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true"></iframe></div>');
}, 6000);



## 6

Run our first test by executing through docker

In [7]:
$$async$$ = true;
var exec = require('child_process').exec;
console.log('running selenium tests');
var runTest = exec('docker exec -t act-selenium npm --prefix /home/seluser/mobile run test', () => {
    $$done$$('test executed');
});
runTest.stdout.on('data', (d) => console.log(d));
runTest.stderr.on('data', (d) => console.log(d));



running selenium tests

> @ test /home/seluser/mobile
> wdio wdio.conf.js




[15:30:32]  COMMAND	POST 	 "/wd/hub/session"

[15:30:32]  DATA		{"desiredCapabilities":{"javascriptEnabled":true,"locationContextEnabled":true,"handlesAlerts":true,"rotatable":true,"maxInstances":5,"browserName":"chrome","chromeOptions":{"args":["no-sandbox","start-fullscreen"]},"loggingPrefs":{"browser":"ALL","driver":"ALL"},"requestOrigins":{"url":"http://webdriver.io","version":"4.6.2","name":"webdriverio"}}}

[15:30:34]  INFO	SET SESSION ID 1d7a8501-be6f-451b-b9d4-da31c6390db7
[15:30:34]  RESULT		{"applicationCacheEnabled":false,"rotatable":false,"mobileEmulationEnabled":false,"networkConnectionEnabled":true,"chrome":{"chromedriverVersion":"2.29.461571 (8a88bbe0775e2a23afda0ceaf2ef7ee74e822cc5)","userDataDir":"/tmp/.org.chromium.Chromium.nwnlLV"},"takesHeapSnapshot":true,"pageLoadStrategy":"normal","databaseEnabled":false,"handlesAlerts":true,"hasTouchScreen":true,"version":"57.0.2987.133","platform":"LIN

'test executed'